In [ ]:
storage_account_name = "tjinsydney"

In [ ]:
blob_container_name = "bde-assignment2"

In [ ]:
dbutils.fs.mount(
  source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
  mount_point = f'/mnt/{blob_container_name}/',
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

Out[5]: True

In [ ]:
dbutils.fs.ls("mnt/bde-assignment2/")

Out[6]: [FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2015.parquet', name='green_taxi_2015.parquet', size=404556105, modificationTime=1726286532000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2016.parquet', name='green_taxi_2016.parquet', size=346731598, modificationTime=1726286513000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2017.parquet', name='green_taxi_2017.parquet', size=251504885, modificationTime=1726286435000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2018.parquet', name='green_taxi_2018.parquet', size=193874396, modificationTime=1726286359000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2019.parquet', name='green_taxi_2019.parquet', size=142163406, modificationTime=1726286267000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2020.parquet', name='green_taxi_2020.parquet', size=37076741, modificationTime=1726286388000),
 FileInfo(path='dbfs:/mnt/bde-assignment2/green_taxi_2021.parquet', name='green_taxi_2021.parquet',

1. Read Parquets files from Azure

In [ ]:
from pyspark.sql import SparkSession

# Read Yellow taxi Parquets 
spark = SparkSession.builder.appName("Parquet Migration").getOrCreate()

files = [
    {"path": "/mnt/bde-assignment2/yellow_taxi_2015.parquet", "year": 2015},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2016.parquet", "year": 2016},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2017.parquet", "year": 2017},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2018.parquet", "year": 2018},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2019.parquet", "year": 2019},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2020.parquet", "year": 2020},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2021.parquet", "year": 2021},
    {"path": "/mnt/bde-assignment2/yellow_taxi_2022.parquet", "year": 2022}
]

for file in files:
    file_path = file["path"]
    year = file["year"]
    
    try:
        
        df = spark.read.parquet(file_path)
        
        destination_path = f"/dbfs/mnt/bde-assignment2/yearly_data/{file_path.split('/')[-1]}"
        
        df.write.mode('overwrite').parquet(destination_path)
        
        print(f"Successfully written {file_path} to {destination_path}")
    except Exception as e:
        print(f"Failed to process {file_path}: {str(e)}")


In [ ]:
from pyspark.sql import SparkSession

# Read Green taxi Parquets 
spark = SparkSession.builder.appName("Parquet Migration").getOrCreate()

files = [
    {"path": "/mnt/bde-assignment2/green_taxi_2015.parquet", "year": 2015},
    {"path": "/mnt/bde-assignment2/green_taxi_2016.parquet", "year": 2016},
    {"path": "/mnt/bde-assignment2/green_taxi_2017.parquet", "year": 2017},
    {"path": "/mnt/bde-assignment2/green_taxi_2018.parquet", "year": 2018},
    {"path": "/mnt/bde-assignment2/green_taxi_2019.parquet", "year": 2019},
    {"path": "/mnt/bde-assignment2/green_taxi_2020.parquet", "year": 2020},
    {"path": "/mnt/bde-assignment2/green_taxi_2021.parquet", "year": 2021},
    {"path": "/mnt/bde-assignment2/green_taxi_2022.parquet", "year": 2022}
]

for file in files:
    file_path = file["path"]
    year = file["year"]
    
    try:
        
        df = spark.read.parquet(file_path)
        
        destination_path = f"/dbfs/mnt/bde-assignment2/yearly_data/{file_path.split('/')[-1]}"
        
        df.write.mode('overwrite').parquet(destination_path)
        
        print(f"Successfully written {file_path} to {destination_path}")
    except Exception as e:
        print(f"Failed to process {file_path}: {str(e)}")


Successfully written /mnt/bde-assignment2/green_taxi_2015.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2015.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2016.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2016.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2017.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2017.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2018.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2018.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2019.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2019.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2020.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2020.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2021.parquet to /dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2021.parquet
Successfully written /mnt/bde-assignment2/green_taxi_2022.parquet to /dbfs/mnt/bde-

2. Combine yearly parquets to one integrated Parquet

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

# Yellow taxi
spark = SparkSession.builder.appName("Parquet Combine").getOrCreate()

files = [
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2015.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2016.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2017.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2018.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2019.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2020.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2021.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2022.parquet"
]

combined_df = None
for file in files:
    try:
        df = spark.read.parquet(file)
        if combined_df is None:
            combined_df = df
        else:
            combined_df = combined_df.union(df)
        print(f"Successfully read {file}")
    except Exception as e:
        print(f"Failed to process {file}: {str(e)}")

In [ ]:
if combined_df is not None:
    destination_path = "/dbfs/mnt/bde-assignment2/yearly_data/combined_yt_data.parquet"
    try:
        combined_df.write.mode('overwrite').parquet(destination_path)
        print(f"Successfully written combined data to {destination_path}")
    except Exception as e:
        print(f"Failed to write combined data: {str(e)}")

In [ ]:
# Green taxi

files2 = [
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2015.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2016.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2017.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2018.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2019.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2020.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2021.parquet",
    "/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2022.parquet"
]

green_df = None
for file in files2:
    try:
        df = spark.read.parquet(file)
        if green_df is None:
            green_df = df
        else:
            green_df = green_df.union(df)
        print(f"Successfully read {file}")
    except Exception as e:
        print(f"Failed to process {file}: {str(e)}")

In [ ]:
if green_df is not None:
    destination_path = "/dbfs/mnt/bde-assignment2/yearly_data/combined_gt_data.parquet"
    try:
        green_df.write.mode('overwrite').parquet(destination_path)
        print(f"Successfully written combined data to {destination_path}")
    except Exception as e:
        print(f"Failed to write combined data: {str(e)}")

In [ ]:
gt_total = spark.read.parquet("/dbfs/mnt/bde-assignment2/yearly_data/combined_gt_data.parquet/")

yt_total = spark.read.parquet("/dbfs/mnt/bde-assignment2/yearly_data/combined_yt_data.parquet/")

In [ ]:
# Green taxi: 66,200,401
gt_total.count()

Out[8]: 66200401

In [ ]:
# Yellow taxi: 663,055,251
yt_total.count()

Out[9]: 663055251

3. Convert the green_taxi_2015 parquet into a .csv and send it to Azure

In [ ]:
gt_2015 = spark.read.parquet("/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2015.parquet/")

In [ ]:
gt_2015.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [ ]:
gt_2015.write.csv("/mnt/bde-assignment2/green_2015#")

In [ ]:
# After merging to one csv file, file size is 2.03gb

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Merge CSV").getOrCreate()

folder_path = "/mnt/bde-assignment2/green_2015/"

df = spark.read.option("header", "true").csv(folder_path)

output_path = "/mnt/bde-assignment2/green_2015/merged_green_taxi_2015.csv"
df.coalesce(1).write.mode('overwrite').option("header", "true").csv(output_path)


4. EDA & Data Engineering 
 - Use 2015 data to explore the dataset and do cleansing integrated dataset

In [ ]:
yellow_2015 = spark.read.parquet("/dbfs/mnt/bde-assignment2/yearly_data/yellow_taxi_2015.parquet/")

green_2015 = spark.read.parquet("/dbfs/mnt/bde-assignment2/yearly_data/green_taxi_2015.parquet/")

In [ ]:
yellow_2015.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, unix_timestamp, when, expr
from pyspark.sql.types import IntegerType, StringType
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
yellow_2015.createOrReplaceTempView("yellow_2015_view")

In [ ]:
spark.sql("""
    SELECT count(*)
    FROM yellow_2015_view
    WHERE tpep_dropoff_datetime < tpep_pickup_datetime
""").show()

+--------+
|count(1)|
+--------+
|    3917|
+--------+



In [ ]:
%sql
SELECT * FROM yellow_2015_view where trip_distance < 0

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2015-04-24T08:27:04.000+0000,2015-04-24T08:29:40.000+0000,1.0,-186318.4,99.0,N,146,7,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3,null,null
1,2015-02-05T07:02:26.000+0000,2015-02-05T07:02:29.000+0000,1.0,-4.08401244E7,1.0,N,132,132,3,2.5,0.5,0.5,0.0,0.0,0.3,3.8,null,null
1,2015-02-07T15:56:13.000+0000,2015-02-07T15:57:03.000+0000,3.0,-4.00955322E7,5.0,N,263,263,1,0.0,0.0,0.0,1.0,0.0,0.3,1.3,null,null


In [ ]:
yellow_2015_with_duration = yellow_2015_with_duration.withColumn("Trip_distance_km", col("Trip_distance") * 1.60934 )

In [ ]:
yellow_2015_with_duration.select("trip_distance", "trip_distance_km").show()

+-------------+------------------+
|trip_distance|  trip_distance_km|
+-------------+------------------+
|         0.42|0.6759227999999999|
|         16.8|         27.036912|
|          8.9|         14.323126|
|         0.75|          1.207005|
|         1.11|         1.7863674|
|         6.57|10.573363800000001|
|         10.0|           16.0934|
|          2.1|          3.379614|
|          1.8|          2.896812|
|          2.5|           4.02335|
|          0.4|0.6437360000000001|
|          0.5|           0.80467|
|         1.84|2.9611856000000003|
|          0.0|               0.0|
|         3.16|         5.0855144|
|         0.67|         1.0782578|
|         8.69|        13.9851646|
|         2.43|3.9106962000000003|
|         1.92|3.0899327999999997|
|         3.58|5.7614372000000005|
+-------------+------------------+
only showing top 20 rows



In [ ]:
yellow_2015_with_duration = yellow_2015.withColumn(
    "trip_duration",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
)

yellow_2015_with_duration.select("trip_duration").summary().show()


+-------+--------------------+
|summary|       trip_duration|
+-------+--------------------+
|  count|           146039231|
|   mean|  19.724179739640576|
| stddev|  26463.156611304476|
|    min|-6.100172596666667E7|
|    max|1.2537316083333333E8|
+-------+--------------------+

+-------+--------------------+
|summary|       trip_duration|
+-------+--------------------+
|  count|           146039231|
|   mean|  19.724179739640576|
| stddev|  26463.156611304476|
|    min|-6.100172596666667E7|
|    25%|   6.633333333333334|
|    50%|  11.066666666666666|
|    75%|  17.933333333333334|
|    max|1.2537316083333333E8|
+-------+--------------------+



In [ ]:
yellow_2015_with_duration = yellow_2015_with_duration.withColumn("speed_kmh", 
                   when(col("trip_duration") > 0, col("trip_distance_km") / (col("trip_duration") / 60)).otherwise(-1))

yellow_2015_with_duration.select("speed_kmh").summary().show()

+-------+--------------------+
|summary|           speed_kmh|
+-------+--------------------+
|  count|           146039231|
|   mean|   -434.187992617175|
| stddev|   6551965.311300155|
|    min|-7.88707749622751...|
|    25%|  12.388881509433963|
|    50%|  16.990099706744868|
|    75%|  23.174495999999998|
|    max| 4.955362957088452E9|
+-------+--------------------+



In [ ]:
yellow_2015_with_duration.createOrReplaceTempView("yellow_2015_view")

In [ ]:
yellow_2015_with_duration.groupBy("passenger_count").count().show()


+---------------+---------+
|passenger_count|    count|
+---------------+---------+
|            8.0|      178|
|            0.0|    40683|
|            7.0|      227|
|            1.0|102929283|
|            4.0|  2980699|
|            3.0|  6133786|
|            2.0| 20893649|
|            6.0|  5123186|
|            5.0|  7937375|
|            9.0|      165|
+---------------+---------+



In [ ]:
yellow_2015_with_duration.groupBy("DOLocationID").count().show()

+------------+-------+
|DOLocationID|  count|
+------------+-------+
|          26|  32255|
|          29|  13891|
|          65| 277527|
|         191|  14085|
|         222|   4609|
|         243| 273943|
|          54|  45855|
|          19|   7895|
|         112| 494328|
|         113|2140592|
|         155|  17175|
|         167|  20870|
|         237|4598356|
|         241|  25815|
|          22|  31592|
|         198|  99332|
|         130|  38608|
|         196|  59141|
|           7| 756019|
|          77|  10470|
+------------+-------+
only showing top 20 rows



In [ ]:
%sql
SELECT *
FROM yellow_2015_view
WHERE DOLocationID > 265 

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_duration,Trip_distance_km,speed_kmh


In [ ]:
yellow_2015_with_duration.select("speed_kmh").summary().show()

In [ ]:
yellow_2015_with_duration.select("trip_distance_km").summary().show()

+-------+------------------+
|summary|  trip_distance_km|
+-------+------------------+
|  count|         146039231|
|   mean|19.063982889507464|
| stddev|17599.690702899785|
|    min|-6.5725645801896E7|
|    25%|           1.60934|
|    50%|         2.7680648|
|    75%| 5.149888000000001|
|    max| 9.4977790010862E7|
+-------+------------------+



In [ ]:
quantiles = yellow_2015_with_duration.approxQuantile("trip_distance_km", [0.9], 0.01)

print(f"90% quantiles: {quantiles[0]}")

90% quantiles: 10.573363800000001


In [ ]:
yellow_2015_with_duration.select("Fare_amount").summary().show()

+-------+------------------+
|summary|       Fare_amount|
+-------+------------------+
|  count|         146039231|
|   mean|12.939035017035645|
| stddev|124.37890358806082|
|    min|            -496.0|
|    25%|               6.5|
|    50%|               9.5|
|    75%|              14.5|
|    max|         825998.61|
+-------+------------------+



In [ ]:
missing_values = yellow_2015_with_duration.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in yellow_2015_with_duration.columns])
missing_values.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------+----------------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_duration|Trip_distance_km|speed_kmh|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------+----------------+---------+
|       0|                   0|                    0|           

In [ ]:
%sql
SELECT * 
FROM (
SELECT AVG(trip_duration) AS avg_duration,
      MAX(trip_duration) AS max_duration,
      MIN(trip_duration) AS min_duration
FROM yellow_2015_view)

avg_duration,max_duration,min_duration
19.724179739640576,1.2537316083333333E8,-6.100172596666667E7


In [ ]:
green_2015 = green_2015.withColumn("Trip_distance_km", col("Trip_distance") * 1.60934)

green_2015.select("Trip_distance_km").summary().show()

+-------+------------------+
|summary|  Trip_distance_km|
+-------+------------------+
|  count|          19233765|
|   mean|4.6349211498355825|
| stddev| 4.855768900844042|
|    min|               0.0|
|    25%|         1.7059004|
|    50%|          3.057746|
|    75%|         5.8579976|
|    max|       1336.395936|
+-------+------------------+



In [ ]:
green_2015.filter(col("Total_amount")< 1).count()

Out[60]: 87877

In [ ]:
green_2015 = green_2015.withColumn(
    "trip_duration",
    (F.unix_timestamp("lpep_dropoff_datetime") - F.unix_timestamp("lpep_pickup_datetime")) / 60
)

green_2015 = green_2015.withColumn("speed_kmh", 
                   when(col("trip_duration") > 0, col("trip_distance_km") / (col("trip_duration") / 60)).otherwise(-1))

green_2015.select("speed_kmh").summary().show()

+-------+------------------+
|summary|         speed_kmh|
+-------+------------------+
|  count|          19233765|
|   mean|25.033145018541312|
| stddev| 290.1030224790009|
|    min|              -1.0|
|    25%|14.884804743083002|
|    50%| 18.77263642458101|
|    75%|23.898699000000004|
|    max|326181.03119999997|
+-------+------------------+



In [ ]:
green_2015.select("trip_duration").summary().show()

+-------+-------------------+
|summary|      trip_duration|
+-------+-------------------+
|  count|           19233765|
|   mean|  17.13734369740654|
| stddev| 1019.9981256241059|
|    min|-483.51666666666665|
|    25%|  6.066666666666666|
|    50%| 10.233333333333333|
|    75%| 16.933333333333334|
|    max|          3155397.5|
+-------+-------------------+



In [ ]:
green_2015.createOrReplaceTempView("green_2015_view")

In [ ]:
%sql
SELECT count(*) FROM green_2015_view WHERE trip_duration > 120

count(1)
66964


Data engineering - Yellow Taxi

In [ ]:
yt_df = yt_total

# Remove trips finishing before the starting time
yt_df = yt_df.filter(col("tpep_dropoff_datetime") > col("tpep_pickup_datetime"))

# Remove Trip_distance is less than 0 
yt_df = yt_df.filter(col("Trip_distance") > 0)

# Remove Fare_amount is less than 0 
yt_df = yt_df.filter(col("Fare_amount") > 0)

# Remove Total_amount is less than 0 
yt_df = yt_df.filter(col("Total_amount") > 0)


In [ ]:
# Add columns for data cleansing

yt_df = yt_df.withColumn("trip_distance_km", col("Trip_distance") * 1.60934)

yt_df = yt_df.withColumn(
    "trip_duration",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
)

yt_df = yt_df.withColumn("speed_kmh", 
                   when(col("trip_duration") > 0, col("trip_distance_km") / (col("trip_duration") / 60)).otherwise(-1))

In [ ]:
# Data cleansing - negative Speed
yt_df = yt_df.filter(col("speed_kmh") >= 0)

# Data cleansing - very high speed
yt_df = yt_df.filter(col("speed_kmh") < 200)

In [ ]:
# Data cleansing - Duration outliers

min_duration = 1
max_duration = 120

# Filter trips based on duration
yt_df = yt_df.filter((yt_df["trip_duration"] >= min_duration) & (yt_df["trip_duration"] <= max_duration))

In [ ]:
# Data cleansing - Distance outliers

min_distance_km = 0.5 
max_distance_km = 80

# Filter trips based on distance_km
yt_df = yt_df.filter((yt_df["trip_distance_km"] >= min_distance_km) & (yt_df["trip_distance_km"] <= max_distance_km))

In [ ]:
yt_df.count()

Out[91]: 645364351

In [ ]:
# Replace passanger_count 0 with mode 1

yt_df = yt_df.withColumn(
    "passenger_count",
    F.when(yt_df["passenger_count"] == 0, 1).otherwise(yt_df["passenger_count"])
)

In [ ]:
yt_df.select("trip_distance_km").summary().show()

+-------+------------------+
|summary|  trip_distance_km|
+-------+------------------+
|  count|         645364351|
|   mean|4.9294506748320055|
| stddev|6.0814579188837525|
|    min|         0.5149888|
|    25%|         1.6576202|
|    50%|         2.7519714|
|    75%| 5.149888000000001|
|    max|         79.984198|
+-------+------------------+



Data engineering - Green Taxi

In [ ]:
gt_df = gt_total

# Remove trips finishing before the starting time
gt_df = gt_df.filter(col("lpep_dropoff_datetime") > col("lpep_pickup_datetime"))

# Remove Trip_distance is less than 0 
gt_df = gt_df.filter(col("Trip_distance") > 0)

# Remove Fare_amount is less than 0 
gt_df = gt_df.filter(col("Fare_amount") > 0)

# Remove Total_amount is less than 0 
gt_df = gt_df.filter(col("Total_amount") > 0)


In [ ]:
# Add columns for data cleansing

gt_df = gt_df.withColumn("trip_distance_km", col("Trip_distance") * 1.60934)

gt_df = gt_df.withColumn(
    "trip_duration",
    (F.unix_timestamp("lpep_dropoff_datetime") - F.unix_timestamp("lpep_pickup_datetime")) / 60
)

gt_df = gt_df.withColumn("speed_kmh", 
                   when(col("trip_duration") > 0, col("trip_distance_km") / (col("trip_duration") / 60)).otherwise(-1))

In [ ]:
# Data cleansing - negative Speed
gt_df = gt_df.filter(col("speed_kmh") >= 0)

# Data cleansing - very high speed
gt_df = gt_df.filter(col("speed_kmh") < 200)

In [ ]:
# Data cleansing - Duration outliers

min_duration = 1
max_duration = 120

# Filter trips based on duration
gt_df = gt_df.filter((gt_df["trip_duration"] >= min_duration) & (gt_df["trip_duration"] <= max_duration))

In [ ]:
# Data cleansing - Distance outliers

min_distance_km = 0.5 
max_distance_km = 100

# Filter trips based on distance_km
gt_df = gt_df.filter((gt_df["trip_distance_km"] >= min_distance_km) & (gt_df["trip_distance_km"] <= max_distance_km))

In [ ]:
gt_df.count()

Out[89]: 63421542

In [ ]:
# Replace passanger_count 0 with mode 1

gt_df = gt_df.withColumn(
    "passenger_count",
    F.when(gt_df["passenger_count"] == 0, 1).otherwise(gt_df["passenger_count"])
)

Combine dataset

 - Manage Yellow & Green taxi dataset before merging
 - Combine Yellow & Green taxi
 - Combine location data
 - Export combined dataset into a parquet file

In [ ]:
yt_df = yt_df.withColumn("taxi_type", F.lit("yellow"))
gt_df = gt_df.withColumn("taxi_type", F.lit("green"))

# Change Column name 
yt_df = yt_df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
yt_df = yt_df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

gt_df = gt_df.withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")
gt_df = gt_df.withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

# Add columns that are not in both datasets and fill them with null values
for column in set(yt_df.columns) - set(gt_df.columns):
    gt_df = gt_df.withColumn(column, F.lit(None).cast(yt_df.schema[column].dataType))

for column in set(gt_df.columns) - set(yt_df.columns):
    yt_df = yt_df.withColumn(column, F.lit(None).cast(gt_df.schema[column].dataType))

combined_df = yt_df.unionByName(gt_df)

combined_df.printSchema()
combined_df.show(5)

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_distance_km: double (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- speed_kmh: double (nullable = true)
 |-- taxi_typ

In [ ]:
# Load the location data 

location_df = spark.read.option("header", True).csv("/FileStore/taxi_zone_lookup.csv")
location_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [ ]:
# Join with the combined dataset using location_id
# Join twice for pickup and dropoff location

combined_df = combined_df.join(
    location_df.withColumnRenamed("LocationID", "PULocationID"),
    on="PULocationID",
    how="left"
)

combined_df = combined_df.withColumnRenamed("borough", "pickup_borough") \
                         .withColumnRenamed("zone", "pickup_zone") \
                         .withColumnRenamed("service_zone", "pickup_service_zone")


combined_df = combined_df.join(
    location_df.withColumnRenamed("LocationID", "DOLocationID"),
    on="DOLocationID",
    how="left"
)

combined_df = combined_df.withColumnRenamed("borough", "dropoff_borough") \
                         .withColumnRenamed("zone", "dropoff_zone") \
                         .withColumnRenamed("service_zone", "dropoff_service_zone")

In [ ]:
combined_df.printSchema()

# Display the first 5 rows of the combined dataframe
combined_df.show(5)

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_distance_km: double (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- speed_kmh: double (nullable = true)
 |-- taxi_typ

In [ ]:
# Export a parquet
destination_path = "/dbfs/mnt/bde-assignment2/nyc_taxi_final.parquet"

try:
    combined_df.write.mode('overwrite').parquet(destination_path)
    print(f"Successfully written combined data to {destination_path}")
except Exception as e:
    print(f"Failed to write combined data: {str(e)}")


Successfully written combined data to /dbfs/mnt/bde-assignment2/nyc_taxi_final.parquet


In [ ]:
destination_path = "/dbfs/mnt/bde-assignment2/nyc_taxi_final.parquet"

nyc_taxi = spark.read.parquet(destination_path)

In [ ]:
nyc_taxi.printSchema()

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_distance_km: double (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- speed_kmh: double (nullable = true)
 |-- taxi_typ

In [ ]:
nyc_taxi.count()

Out[101]: 708785893